In [2]:
import os

import keras

import random

import shutil

import tensorflow as tf

from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from shutil import copyfile

from os import getcwd

from os import listdir

import cv2

from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization,Activation, MaxPooling2D, Flatten, Dense

from tensorflow.keras.models import Model, load_model

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

from sklearn.utils import shuffle

import imutils

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.image  as mpimg

In [3]:
def data_summary(main_path):

    withmask_path = main_path+'withmaskreal'

    withoutmask_path = main_path+'withoutmaskreal'

    # number of (images) that are in the the folder named 'with_mask' 

    m_pos = len(listdir('./observations-master/experiements/data/with_mask'))

    # number of (images) that are in the the folder named 'without_mask' 

    m_neg = len(listdir('./observations-master/experiements/data/without_mask'))

    # number of all examples

    m = (m_pos+m_neg)

    pos_prec = (m_pos* 100.0)/ m

    neg_prec = (m_neg* 100.0)/ m

    print(f"Number of examples: {m}")

    print(f"Percentage of positive examples: {pos_prec}%, number of pos examples: {m_pos}") 

    print(f"Percentage of negative examples: {neg_prec}%, number of neg examples: {m_neg}")    
augmented_data_path = 'facemask-dataset/trial1/augmented data1/'    

data_summary(augmented_data_path)

Number of examples: 1376
Percentage of positive examples: 50.145348837209305%, number of pos examples: 690
Percentage of negative examples: 49.854651162790695%, number of neg examples: 686


In [4]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):

    dataset = []  

    for unitData in os.listdir(SOURCE):

        data = SOURCE + unitData

        if(os.path.getsize(data) > 0):

            dataset.append(unitData)

        else:

            print('Skipped ' + unitData)

            print('Invalid file i.e zero size')  

    train_set_length = int(len(dataset) * SPLIT_SIZE)

    test_set_length = int(len(dataset) - train_set_length)

    shuffled_set = random.sample(dataset, len(dataset))

    train_set = dataset[0:train_set_length]

    test_set = dataset[-test_set_length:]     

    for unitData in train_set:

        temp_train_set = SOURCE + unitData

        final_train_set = TRAINING + unitData

        copyfile(temp_train_set, final_train_set)  

    for unitData in test_set:

        temp_test_set = SOURCE + unitData

        final_test_set = TESTING + unitData

        copyfile(temp_test_set, final_test_set)      

WITHMASK_SOURCE_DIR = "./observations-master/experiements/dest_folder/val/with_mask/"

TRAINING_WITHMASK_DIR = "./observations-master/experiements/dest_folder/train/with_mask/"

TESTING_WITHMASK_DIR = "./observations-master/experiements/dest_folder/test/without_mask/"

WITHOUTMASK_SOURCE_DIR = "./observations-master/experiements/dest_folder/val/without_mask/"

TRAINING_WITHOUTMASK_DIR = "./observations-master/experiements/dest_folder/train/without_mask/"

TESTING_WITHOUTMASK_DIR = "./observations-master/experiements/dest_folder/test/without_mask/"

split_size = .8

split_data(WITHMASK_SOURCE_DIR, TRAINING_WITHMASK_DIR, 

TESTING_WITHMASK_DIR, split_size)

split_data(WITHOUTMASK_SOURCE_DIR, TRAINING_WITHOUTMASK_DIR, 

TESTING_WITHOUTMASK_DIR, split_size)

In [5]:
print("number of images with facemask in the training set'with_mask':", len(os.listdir('./observations-master/experiements//dest_folder//train//with_mask')))

print("number of images with facemask in the test set'with_mask':", len(os.listdir('./observations-master/experiements//dest_folder//test//with_mask')))

print("number of images without facemask in the training set'without_mask':", len(os.listdir('./observations-master/experiements//dest_folder//train//without_mask')))

print("number of images without facemask in the test set'without_mask':", len(os.listdir('./observations-master/experiements//dest_folder//test//without_mask')))

number of images with facemask in the training set'with_mask': 714
number of images with facemask in the test set'with_mask': 97
number of images without facemask in the training set'without_mask': 713
number of images without facemask in the test set'without_mask': 127


In [6]:
model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(100, (3,3), activation='relu', 

    input_shape=(150, 150, 3)),

    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),

    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(50, activation='relu'),

    tf.keras.layers.Dense(2, activation='softmax')

])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [7]:
TRAINING_DIR = "./observations-master/experiements/dest_folder/train"

train_datagen = ImageDataGenerator(rescale=1.0/255,

                                   rotation_range=40,

                                   width_shift_range=0.2,

                                   height_shift_range=0.2,

                                   shear_range=0.2,

                                   zoom_range=0.2,

                                   horizontal_flip=True,

                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, batch_size=10, target_size=(150, 150))



VALIDATION_DIR = "./observations-master/experiements/dest_folder/test"

validation_datagen = ImageDataGenerator(rescale=1.0/255)

 
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 

                                                         batch_size=10, 

                                                         target_size=(150, 150))

checkpoint = ModelCheckpoint('model-{epoch:03d}.model'

,monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

Found 1427 images belonging to 2 classes.
Found 224 images belonging to 2 classes.


In [8]:
history = model.fit_generator(train_generator,

                              epochs=30,

                              validation_data=validation_generator,

                              callbacks=[checkpoint])

model.save('my_model_facemask.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
143/143 [==============================] - ETA: 0s - loss: 0.5158 - acc: 0.7631WARNING:tensorflow:From /home/bhavya/.local/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model-001.model/assets
143/143 [==============================] - 122s 855ms/step - loss: 0.5158 - acc: 0.7631 - val_loss: 0.4978 - val_acc: 0.8393
Epoch 2/30
143/143 [==============================] - 145s 1s/step - loss: 0.3079 - acc: 0.8858 - val_loss: 0.3265 - val_acc: 0.9018
Epoch 3/30
143/143 [=================

In [ ]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()